# This is a gaiamock wrapper, here's how to use it

Ben Pennell

--
_____

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pickle
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 10)

import GaiamockWrapper as gw

In [ ]:
catpath = "data/m_dwarf_gaia_data.pkl"
infile = open(catpath, "rb")
r = pickle.load(infile)
infile.close()

## marginalizing ruwe

You can call `mariginalize_ruwe()` and pass in any number of keywords to then be passed into gaiamock. For every variable, there are default values so you don't need to set everything. The defaults can be seen in `calculate_ruwe()`. If you want to use values for (ra, dec, pmra, pmdec, parallax) from a catalogue, you can pass in a dataframe row to the variable `df`

Let's take an example where we specify a binary and simply see how ruwe varies. I'll leave all the other parameters to defaults (circular, face-on orbit)

In [ ]:
ruwes = gw.marginalize_ruwe(sample_count=1000, df=r[1019], m1=0.4, q=0.1, p=1e4)
plt.hist(ruwes, bins=100, color="red");
plt.xlabel("RUWE");
plt.ylabel("#");

Now, what if we wanted to know how this $\texttt{RUWE}$ distribution changes with different primary mass? You can call `marginalize_ruwe_grid1d` and check. What you need to provide in the first argument is `([paramter name], [list of values])`

In [ ]:
m1s = np.linspace(0.2,0.8,10)
ruwes_1d = gw.marginalize_ruwe_grid1d(("m1", m1s), sample_count=100, df=r[1019], m1=0.4, q=0.1, p=1e4)
plt.errorbar(m1s, np.mean(ruwes_1d, axis=1), yerr=np.std(ruwes_1d, axis=1), color="k")
plt.xlabel("Primary mass");
plt.ylabel("Average RUWE");
plt.title("RUWE distribution dependence on primary mass, std shown as error bar");

Now, we could also check how the $\texttt{RUWE}$ distribution changed with mass AND period by calling `marginalize_ruwe_grid2d`

In [ ]:
m_range = (0.4,1.2)
p_range = (2,7)

m1s = np.linspace(*m_range, 15)
ps = np.logspace(*p_range,15)
ruwes_2d = gw.marginalize_ruwe_grid2d(("m1", m1s), ("period", ps), sample_count=3, df=r[1019], q=0.7)
mean_ruwes_2d = np.mean(ruwes_2d, axis=2)

fig, ax = plt.subplots(1,1)
cb = ax.imshow(mean_ruwes_2d, extent=[p_range[0],p_range[1],m_range[1],m_range[0]], aspect="auto")
plt.colorbar(cb, label="RUWE");
ax.set_ylabel("Primary mass");
ax.set_xlabel("log10 Period");

for i in range(mean_ruwes_2d.shape[0]):
    for j in range(mean_ruwes_2d.shape[1]):
        x_pos = p_range[0] + (p_range[1] - p_range[0]) * (j + 0.5) / mean_ruwes_2d.shape[1]
        y_pos = m_range[0] + (m_range[1] - m_range[0]) * (i + 0.5) / mean_ruwes_2d.shape[0]
        
        ax.text(x_pos, y_pos, f'{mean_ruwes_2d[i, j]:.2f}', ha='center', va='center', color='white')
ax.invert_yaxis()

## Features

So, this was the basic usage. I'll now list the various other features I've included

### marginalization

If you want to marginalize over a particular parameter, you simply supply the function to be called. Currently, I haven't implemented a scheme to pass functions with arguments, but I could do this if there developed a need. So, we could marginalize over inclination with

In [ ]:
def random_inc():
    return np.random.rand()*0.5*np.pi

ruwes = gw.marginalize_ruwe(sample_count=1000, df=r[1019], m1=0.4, q=0.1, p=1e4, inc=random_inc)
plt.hist(ruwes, bins=100, color="red");
plt.xlabel("RUWE");
plt.ylabel("#");
plt.title("Now with marginalized RUWE");

I've created built-in functions for marginalizing over all the orbital angles, and you can tell the program to marginalize all of them by using the keyword argument `marginalize_angles`

In [ ]:
ruwes = gw.marginalize_ruwe(sample_count=1000, df=r[1019], m1=0.4, q=0.1, p=1e4, marginalize_angles=True)
plt.hist(ruwes, bins=100, color="red");
plt.xlabel("RUWE");
plt.ylabel("#");
plt.title("Now with marginalized RUWE");

You can also pass parameters into the functions you're using, if you want to do that, just ask me or look into the code. I'll create documentation for that later

In addition to supplying a function for marginalizing, you can specify to randomly sample from a catalogue. And, if you sample two or more parameters from a catalogue, at each step it only chooses a random row to pull all the parameters from. This prevents unphysical combinations of parameters from showing up.

To do this you need to pass in a `catalogue_path` and then specify the keyword arguments for your parameters as `"catalogue"`

In [ ]:
ruwes = gw.marginalize_ruwe(sample_count=1000, df=r[1019], catalogue_path=catpath,
                            m1=0.4, q=0.1, p=1e4, 
                            ra="catalogue", dec="catalogue")
plt.hist(ruwes, bins=100, color="red");
plt.xlabel("RUWE");
plt.ylabel("#");
plt.title("Now with marginalized RUWE over ra,dec");

there are optional keywords `marginalize_position` and `marginalize_pm` which automatically set (ra,dec) and (pmra,pmdec) to `"catalogue"` respectively. Of course, I have found that pmra and pmdec make little impact on the $\texttt{RUWE}$ distribution, so they can safely be set to 0. For this reason, their default values are zero

If you have a catalogue like the nss catalogue, you may have access to things like the eccentricity. You can tell the program to pull a given parameter from a particular row of the dataframe by passing the row to the keyword `df` and then putting in a keyword with the value `df`

In the same breath, I can mention that you can use the `use_a0` argument to calculate the astrometry using a0 instead of m1,q

In [ ]:
infile_nss = open("data/m_dwarf_nss_data.pkl", "rb")
r_nss = pickle.load(infile_nss)
infile_nss.close()
ruwes = gw.marginalize_ruwe(sample_count=1000, df=r_nss[0], use_a0=True,
                            ecc="df", period="df", a0=0.5)
plt.hist(ruwes, bins=100, color="red");
plt.xlabel("RUWE");
plt.ylabel("#");
plt.title("RUWE distribution for a NSS system");

You can also use the `single_star=True` optional keyword to instead use gaiamock to calculate astrometry for a single star

If you use `marginalize_ruwe_grid1d` you can actually pass as many parameters as you'd like, they will each be computed at the same time. This could be cleverly used with a latin hypercube setup or something intelligent to aleviate the need to do higher order grid searches.

This can let me see what happens if I increase parallax and decrease q at the same time while marginalizing over the binary orientation

In [ ]:
plaxes = np.linspace(10,30,10)
qs = np.linspace(0.8,0.1,10)
ruwes = gw.marginalize_ruwe_grid1d(("parallax", plaxes), ("q", qs), sample_count=1000, df=r[1019], marginalize_angles=True)
plt.plot(qs, np.mean(ruwes, axis=1), c="k")
plt.xlabel("increasing parallax, decreasing q");
plt.ylabel("average RUWE");

Lastly, you can get access to the values of the marginalized parameters at each step by passing in `return_parameters=True`

In [ ]:
ruwes, params = gw.marginalize_ruwe(sample_count=1000, df=r[1019], return_params=True, inc=gw.random_inc)
plt.hist([param["inc"] for param in params], bins=100, color="green");
plt.title("values of inclination");

## Orbital Solutions

in the exact same way that you use the $\texttt{RUWE}$ marginalization functions, you can call functions to calculate the likelihood of an orbital solution to be found. I simplified the orbit solution cascade from `gaiamock` to cut all the extra information and simply return `5`, `7`, `9`, or `12` depending on the number of parameters in the final solution. I've also limited as much as possible when `gaiamock` checks for the full orbit solutions since this nonlinear fit is expensive.

You just need to call

`marginalize_solution_type`

`marginalize_solution_type_grid1d`

`marginalize_solution_type_grid2d`

I'll provide an example. Let's say I want to know how the solution type changes as I move an object away from earth

In [ ]:
plaxes = np.linspace(30,10,10)
solutions = gw.marginalize_solution_type_grid1d(("parallax", plaxes), sample_count=100,
                                           df=r[1019], marginalize_angles=True,
                                           m1=0.8,q=0.3,period=1e4)

binned_solutions = np.array([gw.bin_solutions(solution_set) for solution_set in solutions])

solution_nums = [5,7,9,12]
for i in range(4):
    plt.plot(1000/plaxes, binned_solutions[:,i], label="{}p".format(solution_nums[i]))
plt.xlabel("distance");
plt.ylabel("solution type count");
plt.legend();

Or, interestingly, we can see how the solution type varies with period

In [ ]:
periods = np.logspace(1,8,10)
solutions = gw.marginalize_solution_type_grid1d(("p", periods), sample_count=100,
                                           df=r[1019], marginalize_angles=True,
                                           m1=0.8,q=0.3,parallax=10)

binned_solutions = np.array([gw.bin_solutions(solution_set) for solution_set in solutions])

In [ ]:
solution_nums = [5,7,9,12]
for i in range(4):
    plt.plot(np.log10(periods), binned_solutions[:,i], label="{}p".format(solution_nums[i]))
plt.xlabel("logPeriod");
plt.ylabel("catalogue solution rate");
plt.legend();

there's actually a lot of other little things I've put in here but don't have the time to write the documentation for. Basically, if there's something more exotic that you're wanting to do, just let me know and I can make it happen. Or look into the code. It's somewhat commented

#### passing parameters into marginalisation functions

In [ ]:
ruwes, params = gw.marginalize_ruwe(df=r[1019], sample_count=1000, return_params=True, parallax=(gw.sample_from_normal, (33, 10)))

In [ ]:
plt.hist([p["parallax"] for p in params])